In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import pandas as pd
from tensorflow.keras.applications import ResNet50V2

tf.__version__

'2.3.0-dev20200522'

In [ ]:
PATH = 'data'

os.listdir(PATH)

['jpeg224',
 'sample_submission.csv',
 'subset.csv',
 'test.csv',
 'train.csv',
 'train_split.csv',
 'val_split.csv']

In [ ]:
train = pd.read_csv(f'{PATH}/train_split.csv')
#train = pd.read_csv(f'{PATH}/subset.csv')
val = pd.read_csv(f'{PATH}/val_split.csv')

train.shape, val.shape

((22194, 8), (10932, 8))

In [ ]:
train.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_6147712,IP_8210115,male,55.0,torso,unknown,benign,0
1,ISIC_8460669,IP_1246698,female,40.0,lower extremity,unknown,benign,0
2,ISIC_3272999,IP_3512888,female,35.0,torso,nevus,benign,0
3,ISIC_2195125,IP_4576977,male,45.0,torso,unknown,benign,0
4,ISIC_8557006,IP_2950485,female,40.0,lower extremity,unknown,benign,0


In [ ]:
BATCH_SIZE = 64
AUTO = tf.data.experimental.AUTOTUNE

def decode(filename, label):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32)
    return img, label

def load_ds(df):
    options = tf.data.Options()
    options.experimental_deterministic = False
    imgs, labels = df["image_name"].values, df["target"].values
    imgs = [f'{PATH}/jpeg224/train/{img}.jpg' for img in imgs]
    ds = tf.data.Dataset.from_tensor_slices((imgs, labels))
    ds = ds.with_options(options)
    ds = ds.map(decode, num_parallel_calls=AUTO)
    ds = ds.cache()
    ds = ds.shuffle(2048)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTO)
    return ds

In [ ]:
train_ds = load_ds(train)
val_ds = load_ds(val)

In [ ]:
IMAGE_SIZE = (224, 224, 3)

encoder = ResNet50V2(
    include_top=False,
    input_shape=IMAGE_SIZE,
    weights='imagenet'
)
encoder.trainable = False

inputs = keras.Input(shape=IMAGE_SIZE)
x = keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)
x = encoder(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
rescaling (Rescaling)        (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 23,566,849
Trainable params: 2,049
Non-trainable params: 23,564,800
_________________________________________________________________


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.AUC(name="auc")]
)

In [ ]:
filepath = './checkpoints/checkpoint'
cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = filepath,
    monitor="val_auc",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="max"
)

In [ ]:
model.fit(
    train_ds, 
    epochs=10, 
    validation_data=val_ds, 
    #validation_steps=10,
    callbacks=[cb]
)

Epoch 1/10
347/347 [==============================] - ETA: 0s - loss: 0.0922 - auc: 0.6820
Epoch 00001: val_auc improved from -inf to 0.79516, saving model to ./checkpoints\checkpoint
347/347 [==============================] - 95s 274ms/step - loss: 0.0922 - auc: 0.6820 - val_loss: 0.0767 - val_auc: 0.7952
Epoch 2/10
347/347 [==============================] - ETA: 0s - loss: 0.0743 - auc: 0.8100
Epoch 00002: val_auc improved from 0.79516 to 0.83388, saving model to ./checkpoints\checkpoint
347/347 [==============================] - 89s 258ms/step - loss: 0.0743 - auc: 0.8100 - val_loss: 0.0746 - val_auc: 0.8339
Epoch 3/10
347/347 [==============================] - ETA: 0s - loss: 0.0709 - auc: 0.8423
Epoch 00003: val_auc did not improve from 0.83388
347/347 [==============================] - 89s 258ms/step - loss: 0.0709 - auc: 0.8423 - val_loss: 0.0740 - val_auc: 0.8251
Epoch 4/10
347/347 [==============================] - ETA: 0s - loss: 0.0674 - auc: 0.8619
Epoch 00004: val_auc impr

In [ ]:
model.load_weights(filepath)
model.save("model.h5")

In [ ]:
model.evaluate(val_ds)

171/171 [==============================] - 29s 172ms/step - loss: 0.0729 - auc: 0.8400


[0.07287885993719101, 0.8400139212608337]

In [ ]:
test = pd.read_csv(f'{PATH}/test.csv')

test.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052060,IP_3579794,male,70.0,NaN
1,ISIC_0052349,IP_7782715,male,40.0,lower extremity
2,ISIC_0058510,IP_7960270,female,55.0,torso
3,ISIC_0073313,IP_6375035,female,50.0,torso
4,ISIC_0073502,IP_0589375,female,45.0,lower extremity


In [ ]:
def decode_test(filename):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32)
    return img

def load_test_ds(df):
    imgs = df["image_name"].values
    imgs = [f'{PATH}/jpeg224/test/{img}.jpg' for img in imgs]
    ds = tf.data.Dataset.from_tensor_slices(imgs)
    ds = ds.map(decode_test)
    ds = ds.batch(BATCH_SIZE)
    return ds

In [ ]:
test_ds = load_test_ds(test)

In [ ]:
preds = model.predict(test_ds)

In [ ]:
preds

array([[0.00370706],
       [0.00145345],
       [0.00221092],
       ...,
       [0.05133142],
       [0.00280571],
       [0.00054312]], dtype=float32)

In [ ]:
submission = pd.DataFrame({'image_name': test['image_name'].values, 'target': preds.ravel()})

submission

,image_name,target
0,ISIC_0052060,0.003707
1,ISIC_0052349,0.001453
2,ISIC_0058510,0.002211
3,ISIC_0073313,0.000887
4,ISIC_0073502,0.000169
...,...,...
10977,ISIC_9992485,0.007189
10978,ISIC_9996992,0.090941
10979,ISIC_9997917,0.051331
10980,ISIC_9998234,0.002806


In [ ]:
submission.to_csv('submission.csv', index=False)